In [10]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from typing import List, Tuple
import pickle

In [11]:
import sys
sys.path.append('../../../../')

In [12]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

In [13]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [14]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

In [15]:
from model import build_model

In [16]:
model = build_model([[256, 256]], 0.2, 
                    [[256, 256]], 0.2, 
                    [[256, 256]], 0.2,
                    'relu', (10, 9), 10, 256, 256, 8)

initializing projection for skip connection: 11 -> 256


In [17]:
res = model([input_data, coordinates, size_value, n_patches_hot_encoded_value])
res

<tf.Tensor: shape=(76632, 1), dtype=float32, numpy=
array([[0.8752438 ],
       [0.78156495],
       [0.8107893 ],
       ...,
       [0.67460513],
       [0.6874007 ],
       [0.78612494]], dtype=float32)>

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 number_of_patches_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['number_of_patches_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[0][0]']         
 ingOpLambda)                                                                                     
                                                                                              

## playground

In [8]:
from model import create_broadcasted_features, apply_mlps, mask_inputs
from models.patch_to_score.models.patch_attention.modules.global_sum_pooling import GlobalSumPooling
from models.patch_to_score.models.patch_attention.modules.patch_attention_with_pair_bias import PatchAttentionWithPairBias
from models.patch_to_score.models.patch_attention.modules.transformer_encoder_mlp import TransformerEncoderMLP
from models.patch_to_score.models.patch_attention.modules.pairs_transition import PairsTransition

In [23]:
features_mlp_hidden_sizes = attention_mlp_hidden_sizes = output_mlp_hidden_sizes = [[256, 256]]
features_mlp_dropout_rate = attention_mlp_dropout_rate = output_mlp_dropout_rate = 0.2
activation = 'relu'
pairs_channel_dimension = attention_dimension = 256
num_heads = 8
 
# [[256, 256]], 0.2, 
# [[256, 256]], 0.2,
# 'relu', (10, 9), 10, 256, 256, 8

In [38]:
mask_condition = tf.cast(tf.keras.layers.Masking(
        mask_value=0)(input_data)._keras_mask, tf.float32)
features = create_broadcasted_features(
    n_patches_hot_encoded_value, max_number_of_patches, size_value, input_data)
pairwise_distances = tf.norm(
    tf.expand_dims(coordinates, axis=1) - tf.expand_dims(coordinates, axis=2), axis=-1)
pairwise_distances = tf.cast(pairwise_distances, tf.float32)

# zero out broadcased features where mask is 0
features = features * mask_condition[..., None]
pairwise_distances = pairwise_distances * mask_condition[..., None]

features, pairwise_distances = mask_inputs(features, pairwise_distances)

F = apply_mlps(features, features_mlp_hidden_sizes,
                features_mlp_dropout_rate, activation)

pairs_transition = PairsTransition(pairs_channel_dimension)
D = pairs_transition(pairwise_distances)

attention_output = PatchAttentionWithPairBias(
    attention_dimension, num_heads)([F, D])

attention_mlp_output = apply_mlps(
    attention_output, attention_mlp_hidden_sizes, attention_mlp_dropout_rate, activation)

global_pooling_output = GlobalSumPooling(
    data_format='channels_last')(attention_mlp_output)

output_mlp_output = apply_mlps(
    global_pooling_output, output_mlp_hidden_sizes, output_mlp_dropout_rate, activation)

output = tf.keras.layers.Dense(
    1, activation='sigmoid')(output_mlp_output)

initializing projection for skip connection: 11 -> 256


2025-06-02 18:12:44.646426: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at strided_slice_op.cc:105 : INVALID_ARGUMENT: Multiple ellipses in slice spec not allowed


In [35]:
output

<tf.Tensor: shape=(76632, 1), dtype=float32, numpy=
array([[0.5533056 ],
       [0.619216  ],
       [0.5120182 ],
       ...,
       [0.41534168],
       [0.60217166],
       [0.72498924]], dtype=float32)>